In [ ]:
import string
from stemming.porter2 import stem
from textblob import TextBlob
import sys  
import math
from collections import Counter
import os
import operator
from random import shuffle
import nltk
import statistics
from sklearn import metrics
import scipy
from string import punctuation
import numpy
from statsmodels.stats.power import TTestIndPower
import random
# fix UTF8 errors
reload(sys)  
sys.setdefaultencoding('utf8')

'''

The following file contains helper code for the World Bank project on understanding internal innovation.

Major functionality found in this file:

1. Parsing and pre-processing project documentation (unstructured text)

2. Extracting text-based features from project documentation

3. Computing correlations and running t-tests over project features between innovative and non-innovative documents

4. Fitting a predictive model to classify projects on is_innovative based on text features

5. Methods to assist in creating dictionaries based on word frequencies from a set of innovative and a set of non-innovative documents

'''


techList = ["startup", "entrepreneurship", "reconstruct", "wind", "talent", "prototype", "mentorship", "hydro",
"ventur", "electrif", "internship", "tech", "technic", "technolog", "incub", "batch", "internet", "lab", "manufacturer", 
"solar", "air", "renewab", "acceler", "hub", "rd", "invest", "car", "renew", "mobil", "research", "network", "onlin", "pilot", 
"msme", "mfis", "it", "e-govern", "om", "excel", "surveil", "ict", "innov", "energi", "experi", "test", "progress", "introduc"]

peopleList = ["governor", "mentorship", "angel","trader","manufacturer","mentor","entrepreneur","secretariat",
"internship","company","graduate","trainer","supplier","auditor","youth","master","member","team","exchange","coach","committee","forum",
"institut","institution","council","offici","student","collaborate","discuss","faculti","workshop","lab","centre","agency","consult",
"particip","assist","student","depend","societi","dissemin","associ","outreach","guid","person","share","competit","firm",
"supervis","author","engag"]

locationList = ["countri", "market", "counti", "citi", "govern", "sector", "region", "state", "network", "agenc", "municip", "infrastructure",
"intern", "local", "urban", "rural"]

modifierList = ["minimum", "sound","resilient","simple","solid","good","viable","proper","light","firm","minor","suitable","feasible",
"veri","full","least","small","averag","specif","potenti","onli","possibl","more","additional","main","over","basic","high","direct","limit","various","least"]
actVerbList = ["implement","plan","oper","evalu","work","build","transport","perform","integr",
"strengthen","coordin","construct","impact","audit","exercise","design","procur","establish","conduct","start"]

timeList =["year","month","week", "day","hour","time","spring","summer","fall","winter","january","february","march",
"april", "may","june","july","august","september","october","november","december", "annual", "yearly", "monthly", "weekly", "daily", "biweekly",
"last", "first", "near", "immediate", "quick", "earli", "gradual", "rapid", "initi", "futur", "regular", "second", "third", "new", "frequent"]

financeList = ["financ","cost","procur","grant","fund","financi","loan","alloc","revenue","menu","bid","spend","economi",
"equity","lend","invest","ida","capit","account","credit"]

jargonList = ["support","develop","manag","provid","improv","inform","monitor","process","requir","assess","result","effect",
"prepar","report","target","identifi","strategi","resource","key","ensure","data","measur","staff","organ","review",
"facil","issue","carry","approach","expect","indic","effici","practic","procedur","detail","respons"]

listOfWordLists = [["actVerb", actVerbList], ["modifier", modifierList], ["location", locationList], ["finance", financeList], ["people", peopleList], ["tech", techList],  ["time", timeList], ["jargon", jargonList]]

sentenceTokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Get sentence length, punctuation, section length, part-of-speech, etc.
def getPunctAndPOSFeatures(fileStr):
	# POD features
	# reference: https://www.garysieling.com/blog/part-of-speech-tagging-nltk-vs-stanford-nlp
	text = nltk.word_tokenize(fileStr)
	tagged = nltk.pos_tag(text)
	pos = Counter([y[1] for y in tagged])
	pos = Counter({k: v / float(len(tagged)) for k, v in pos.items()})
	# Punctuation features
	sentences = sentenceTokenizer.tokenize(fileStr)
	wordsInSentences = [len(nltk.word_tokenize(sentence)) for sentence in sentences]
	wordsPerSentence = float(sum(wordsInSentences))/len(wordsInSentences)
	wordsInSection = sum(wordsInSentences)
	punctuation_counts =  {k:v/float(sum(wordsInSentences)) for k, v in Counter(fileStr).iteritems() if k in punctuation}
	return wordsPerSentence, wordsInSection, punctuation_counts, pos

# Get abbreviation features
def countAbbrevs(fileStr):
	#tokenize words
	words = fileStr.split()
	#find all caps words
	abbrevs = [x for x in words if x.isupper() and not any(char.isdigit() for char in x) and len(x) > 1]
	#num abbrevs, num unique abbrevs
	return len(abbrevs), len(set(abbrevs))

# Extract risk features (avg risk score, # high/substantial ratings, etc.) from risk doc
def getHMLAvg(text):
	words = text.split()
	isH = [x for x in words if x == "h" or x == "high"]
	isS = [x for x in words if x == "s" or x == "substantial"]
	isM = [x for x in words if x == "m" or x == "moderate" or x == "medium"]
	isL = [x for x in words if x == "n" or x == "l" or x == "negligible" or x == "low"]
	risk = len([x for x in words if x == "risk" or x == "risks"])
	numRatings = len(isH + isS + isM + isL)
	if numRatings > 0:
		avgRating = (3.0 * len(isH) + 2.0 * len(isS) + 1.0 * len(isM))/numRatings
		high = float(len(isH))/numRatings
	else:
		avgRating = 1.0
		high = 0.0
	print numRatings, avgRating, high, risk
	return numRatings, avgRating, high, risk

# Count frequencies for words in different dictionaries (time-based, tech, etc.)
def countWordsForBaseline(fileStr):
	fileStr = fileStr.translate(None, string.punctuation)
	fileStr = fileStr.lower()
	fileStr = unicode(fileStr, errors='ignore')
	myDict = fileStr.split()
	myDict = [stem(word) for word in myDict]
	allCatsFreqs = {}
	for category in listOfWordLists:
		catWordFreqs = {}
		wordList = [stem(word) for word in category[1]]
		for word in wordList:
			catWordFreqs[word] = float(myDict.count(word)) / len(myDict)
		allCatsFreqs[category[0]] = catWordFreqs
	return allCatsFreqs

# Count frequencies of words in train/test documents and compare to the word frequency baselines
def countWordsAndCompareToBaseline(fileStr, fileType, baselines):
	myDict = fileStr.split()
	myDict = [stem(word) for word in myDict]
	allCatsFreqs = {}
	for category in listOfWordLists:
		catWordFreqs = {}
		wordList = [stem(word) for word in category[1]]
		for word in wordList:
			if fileType in baselines[category[0]][word]:
				baselineWord = baselines[category[0]][word][fileType]
				if baselineWord != 0.0:
					# Compute ratio between document word frequency and baseline frequency of this word
					catWordFreqs[word] = (float(myDict.count(word)) / len(myDict)) / baselineWord
		allCatsFreqs[category[0]] = catWordFreqs
	return allCatsFreqs

def safeCheck(dic, word):
	return dic[word] if word in dic else 0.0

def getFeaturesFromComponents(features):
	return [features['numUniqueAbbrevs'], features['numAbbrevs'], features['sentiment'].polarity, features['sentiment'].subjectivity, features['wordsPerSentence'], features['wordsInSection'], safeCheck(features['punctuationCounts'], '$'), 
	safeCheck(features['punctuationCounts'], ')'), safeCheck(features['punctuationCounts'], '-'),safeCheck(features['punctuationCounts'], ','), 
	safeCheck(features['punctuationCounts'], ':'), features['actVerb'], features['modifier'], features['location'], features['finance'], features['people'], features['tech'], features['time'], features['jargon'],
	(safeCheck(features['pos'], 'NN') + safeCheck(features['pos'], 'NNP') + safeCheck(features['pos'], 'NNS')), safeCheck(features['pos'], 'IN'), safeCheck(features['pos'], 'JJ'),
	safeCheck(features['pos'], 'DT'), safeCheck(features['pos'], 'CC'), safeCheck(features['pos'], 'MD'), safeCheck(features['pos'], 'RB'), 
	(safeCheck(features['pos'], 'VB') + safeCheck(features['pos'], 'VBN') + safeCheck(features['pos'], 'VBG') + safeCheck(features['pos'], 'VBZ') + safeCheck(features['pos'], 'VBP')),
	safeCheck(features['pos'], 'CD'), safeCheck(features['pos'], 'TO')]

def getFeaturesFromPDO(features):
	return [features['wordsInSection']]

def getFeaturesFromRisk(features):
	return [features['sentiment'].polarity, features['sentiment'].subjectivity, features['numRisks'], features['avgRisk'], features['highRisk'], features['risks']]

# Generate set of features for classification modeling
def getFullFeatures(comp, pdo, risk):
	finalFeatures = []
	for proj, features in comp.iteritems():
		if proj in pdo and proj in risk:
			finalFeatures.append(getFeaturesFromComponents(comp) + getFeaturesFromPDO(dict2[pdo]) + getFeaturesFromRisk(dict3[risk]))
	# randomize order of results
	shuffle(finalFeatures)
	return finalFeatures

# Compute all features for a given document section (components, risk, PDO)
def featureExtractionHelper(baselineDir, nonInnovDir, innovDir, pcodesForExtraction, isPdo, isRisk):
	# Get baseline
	baselineDicts = transformDocToDict(baselineDir, None, countWordsForBaseline)
	baselineDictsWithFileTypes = [[projInfo[7].lower()] + processed for processed in baselineDicts for projInfo in pcodesForExtraction if processed[0]==projInfo[0]]
	baselines = computeAvgWordFreqOverDict(baselineDictsWithFileTypes)
	### Evaluate
	print "INNOV"
	evaluate_innov = evaluateDocs(innovDir, pcodesForExtraction, baselines, True, isPdo, isRisk)
	innov_long = {x['name']:x for x in evaluate_innov if x['fileType'] == "long"}
	# Non innov projs from non innov ttls
	print "NON-INNOV"
	evaluate_non_innov = evaluateDocs(nonInnovDir, pcodesForExtraction, baselines, True, isPdo, isRisk)
	non_innov_long =  {x['name']:x for x in evaluate_non_innov if x['fileType'] == "long"}
	return innov_long, non_innov_long

# Run t-test and compute correlation coefficients between features and is_innov
def computeFeatSignificance(innov, non_innov):
	significance = []
	for featureIndex in range(len(innov[0])):
		capped_innov_feature_values = removeAndCapOutliers([x[featureIndex] for x in innov])
		capped_non_innov_feature_values = removeAndCapOutliers([x[featureIndex] for x in non_innov])
		corrCoeff = scipy.stats.pearsonr(capped_innov_feature_values + capped_non_innov_feature_values, [1.0] * len(capped_innov_feature_values) + [0.0] * len(capped_non_innov_feature_values))
		t_test= scipy.stats.ttest_ind(capped_innov_feature_values, capped_non_innov_feature_values, axis = 0, equal_var=False)
		significance.append([corrCoeff, t_test])
	return significance

def transformDocToDict(inputDir, extractFn = None, processFn = None):
	listOfDicts = []
	for filename in os.listdir(inputDir):
		print filename
		f = open(inputDir + "/" + filename, 'r')
		section =  f.read().lower()
		docDict = processFn(section)
		listOfDicts.append([filename[0:7], docDict])
	return listOfDicts

# Function to get all features from a document
def evaluateDocs(inputDir, pcodes, baselines, getDictFeatures = True, isPdo = False, isRisk = False):
	docFeatures = []
	sentenceTokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
	for filename in os.listdir(inputDir):
		features = {'name': filename[0:7]}
		print filename
		f = open(inputDir + "/" + filename, 'r')
		fileStr =  f.read()

		# Get rid of troublesome characters
		fileStr = unicode(fileStr, errors='ignore')
		if isPdo:
			fileStr = sentenceTokenizer.tokenize(fileStr)
			fileStr = fileStr[0]
		wordsPerSentence, wordsInSection, punctuation_counts, pos = getPunctAndPOSFeatures(fileStr)
		features['wordsPerSentence'] = wordsPerSentence
		features['wordsInSection'] = wordsInSection
		features['punctuationCounts'] = punctuation_counts
		features['pos'] = pos

		# Remove punctuation
		exclude = set(string.punctuation)
		fileStr = ''.join(ch for ch in fileStr if ch not in exclude)

		# Count abbrevs
		numAbbrevs, numUniqueAbbrevs = countAbbrevs(fileStr)
		features['numAbbrevs'] = numAbbrevs
		features['numUniqueAbbrevs'] = numUniqueAbbrevs

		# Lower case 
		fileStr = fileStr.lower()

		# Get sentiment
		features['sentiment']  = TextBlob(fileStr).sentiment
		fileType = [x[7] for x in pcodes if x[0] == filename[0:7]][0].lower()

		# Dict features
		if getDictFeatures:
			if fileType == "pad" or fileType == "project paper":
				features['fileType'] = "long"
				features['dictFeatures'] = countWordsAndCompareToBaseline(fileStr, "long", baselines)
			else:
				features['fileType'] = "short"
				features['dictFeatures'] = countWordsAndCompareToBaseline(fileStr, "short", baselines)
		for cat, catDict in features['dictFeatures'].iteritems():
			if len(catDict.values()) > 0:
				catMean = sum(catDict.values())/float(len(catDict.values()))
				features[cat] = catMean
		if isRisk:
			numRisks, avgRisk, highRisk, risks = getHMLAvg(fileStr)
			features['numRisks'] = numRisks
			features['avgRisk'] = avgRisk
			features['highRisk'] = highRisk
			features['risks'] = risks
		docFeatures.append(features)
	return docFeatures

# Use this method to compute baselines for each category, filetype pair (e.g. 'time', "LONG")
# Filetype options are LONG, SHORT
# PAD includes PAD and proj paper
# non-PAD includes PID, PCN, and misc.
def computeAvgWordFreqOverDict(listOfDicts):
	dict0 = listOfDicts[0][2]
	allCatMeans = {}
	for k, v in dict0.iteritems():
		oneCatMean = {}
		allDictsForCat = [[x[2][k], x[0]] for x in listOfDicts]
		longWordMeans = []
		shortWordMeans = []
		for kWord, vWord in allDictsForCat[0][0].iteritems():
			for currDict in allDictsForCat:
				if currDict[1] == "pad" or currDict[1] == "project paper":
					longWordMeans.append(currDict[0][kWord])
				else:
					shortWordMeans.append(currDict[0][kWord])
			meanLongWordFreq = sum(longWordMeans)/len(longWordMeans)
			meanShortWordFreq = sum(shortWordMeans)/len(shortWordMeans)
			oneCatMean[kWord] = {'long': meanLongWordFreq, 'short': meanShortWordFreq}
		allCatMeans[k] = oneCatMean
	return allCatMeans

# Cap outliers beyond mean +- 3*SD at those thresholds
def removeAndCapOutliers(dataList):
	mean = statistics.mean(dataList)
	sd = statistics.stdev(dataList)
	upperCutoff = mean + 3 * sd
	lowerCutoff = mean - 3*sd
	finalList = []
	for x in dataList:
		if x > upperCutoff:
			finalList.append(upperCutoff)
		elif x < lowerCutoff:
			finalList.append(lowerCutoff)
		else:
			finalList.append(x)
	return finalList

### The following methods help with PAD and PID component description extraction
def extractPADInfo(fileStr):
	annexStartInd = fileStr.rfind(": detailed project description")
	if annexStartInd > 0:
		annexNumber = fileStr[fileStr.rfind(" ", 0, annexStartInd-1) + 1 : annexStartInd]
		nextAnnexStartInd = fileStr.rfind("annex " + str(int(annexNumber) + 1) + ":")
	else:
		nextAnnexStartInd = -1
	return annexStartInd, nextAnnexStartInd

def extractPIDInfo(fileStr):
	descripStartInd = max(fileStr.rfind(". Preliminary Description"), fileStr.rfind(". Description"), fileStr.rfind(". Project Description"),)
	nextDescripSectionInd = -1
	if descripStartInd > 0:
		descripSection = fileStr[fileStr.rfind(" ", 0, descripStartInd-1) + 1 : descripStartInd]
		descripSection = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', descripSection)
		if "\n" in descripSection:
			descripSection = descripSection[descripSection.rfind("\n")+1:]
		if descripSection == 'III':
			nextDescripSectionInd = fileStr.rfind("IV. ")
		elif descripSection == 'II': 
			nextDescripSectionInd = fileStr.rfind("III. ")
		elif descripSection == 'IV':
			nextDescripSectionInd = fileStr.rfind("V. ")
		elif descripSection.isdigit():
			nextDescripSectionInd = fileStr.rfind(str(int(descripSection) + 1) + ". ") 
		else:
			print "could not find next section; previous section called ", descripSection
	return descripStartInd, nextDescripSectionInd

# The following methods help generate and combine dictionary data in order to support the manual dictionary curation process
def combinedDicts(listOfDicts):
	print "combining document dictionaries"
	fullDict = {}
	for currDict in listOfDicts:
		for k, v in currDict.iteritems():
			if k in fullDict:
				fullDict[k].append(v)
			else:
				fullDict[k] = [v]
	fullDict = {k: v + [0.0] * (len(listOfDicts) - len(v)) for k, v in fullDict.items()}
	return fullDict
	
def compareDicts(innovDict, nonInnovDict):
	# Do t-tests to compare freqs of of any word that appears at least once in innov Dict
	# Get # of docs each corpus
	print "comparing full dictionaries"
	numDocsNonInnov = len(nonInnovDict['the'])
	numDocsInnov = len(innovDict['the'])
	compareDictResults = {}
	powerCalc = TTestIndPower()
	for k, v in innovDict.iteritems():
		innovFreqs = innovDict[k]
		if k in nonInnovDict:
			nonInnovFreqs = nonInnovDict[k]
		else:
			nonInnovFreqs = [0.0] * numDocsNonInnov
		# Get max freqs
		maxInnovFreq = max(innovFreqs)
		maxNonInnovFreq = max(nonInnovFreqs)
		# T test, 2 sample unequal variance
		t_test = scipy.stats.ttest_ind(innovFreqs, nonInnovFreqs, axis=0, equal_var=False)
		# Power calculation, 2 sample t test with pooled std. get minimal detectable effect
		mde = powerCalc.solve_power(effect_size=None, nobs1=numDocsInnov, alpha=0.05, power=0.8, ratio= float(numDocsNonInnov)/numDocsInnov, alternative='two-sided') * pooledStd(innovFreqs, nonInnovFreqs)
		compareDictResults[k] = {'meanInnovWordFreq': sum(innovFreqs) / len(innovFreqs), 'fracOfInnovDocsAppearing': sum(x > 0 for x in innovFreqs) / (0.0 + numDocsInnov), 
								'meanNonInnovWordFreq': sum(nonInnovFreqs) / len(nonInnovFreqs), 'fracOfNonInnovDocsAppearing': sum(x > 0 for x in nonInnovFreqs) / (0.0 + numDocsNonInnov),
								'innovMinusNonInnovFreq': sum(innovFreqs) / len(innovFreqs) - sum(nonInnovFreqs) / len(nonInnovFreqs),
								'p_val': t_test[1], 'mde': mde, 'maxInnovFreq': maxInnovFreq, 'maxNonInnovFreq': maxNonInnovFreq, 'maxInnovMinusNonInnovFreq': maxInnovFreq - maxNonInnovFreq}
	sorted_compare = sorted(compareDictResults.items(), key=lambda t: -1 * t[1]['meanInnovWordFreq'])
	return sorted_compare

def pooledStd(arr1, arr2):
	return math.sqrt(((len(arr1) - 1) * numpy.var(arr1) + (len(arr2) - 1) * numpy.var(arr2)) / (len(arr1) + len(arr2) - 2))

def main():
	pcodes = [line.rstrip('\n').split('\t') for line in open('pcodes.txt')]
	cats = [x[0] for x in listOfWordLists]
	# COMPONENTS
	innov_long_components, non_innov_long_components = featureExtractionHelper("FULL/components/non_innov_non_innov_TTL/baseline", "FULL/components/non_innov_non_innov_TTL/test",
	"FULL/components/innov/", pcodes, isPdo = False, isRisk = False)
	# PDO
	innov_long_PDO, non_innov_long_PDO = featureExtractionHelper("FULL/PDO/non_innov_non_innov_TTL/baseline", "FULL/PDO/non_innov_non_innov_TTL/test",
		"FULL/PDO/innov/", pcodes, isPdo = True, isRisk = False)
	# Risk
	innov_long_risk, non_innov_long_risk = featureExtractionHelper("FULL/risk/non_innov_non_innov_TTL/baseline", "FULL/risk/non_innov_non_innov_TTL/test",
		"FULL/risk/innov/", pcodes, isPdo = False, isRisk = True)

	confusion_matrix = []
	accuracies = []
	for i in range(10):
		# Get full features
		innov_final_features = getFullFeatures(innov_long_components, innov_long_PDO, innov_long_risk)#, innov_long_risk)
		non_innov_final_features = getFullFeatures(non_innov_long_components, non_innov_long_PDO, non_innov_long_risk)#, non_innov_long_risk)
		computeFeatSignificance(innov_final_features, non_innov_final_features)
		# Model
		from sklearn.linear_model import LogisticRegression
		features_train = innov_final_features[0:int(0.5*len(innov_final_features))] + non_innov_final_features[0:int(0.5*len(non_innov_final_features))] 
		features_test = innov_final_features[int(0.5*len(innov_final_features)):] + non_innov_final_features[int(0.5*len(non_innov_final_features)):] 
		labels_train = [1.0] * int(0.5*len(innov_final_features)) + [0.0] * int(0.5*len(non_innov_final_features))
		labels_test = [1.0] * (len(innov_final_features) - int(0.5*len(innov_final_features))) + [0.0] * (len(non_innov_final_features) - int(0.5*len(non_innov_final_features)))
		model = LogisticRegression()
		model = model.fit(features_train, labels_train)
		prediction_lr = model.predict(features_test)
		confusion_matrix.append(metrics.classification_report(labels_test, prediction_lr))
		accuracies.append(metrics.accuracy_score(prediction_lr, labels_test))
